# Analysis Flow

This notebook demonstrates how an Automate Flow can be used to perform analysis. 
We use a flow that combines Globus Transfer and funcX to move data and then execute an analysis code on the data.

In [1]:
from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)

import time
import json
import sys
import os

from funcx.sdk.client import FuncXClient

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

## Create an analysis function

Use funcX to register a simple analysis function.

In [2]:
fxc = FuncXClient()

In [3]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    from subprocess import PIPE
    
    python_path = data['python_path']
    script_path = data['script_path']
    
    #convert below params similar to rec_upd_pos
    ifpath = data['ifpath']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_ngpu = data['rec_ngpu']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    
    try:
        os.mkdir(ofpath)
    except:
        pass
    
    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' --folder='{ofpath}'"
    
    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr
    
func_uuid = fxc.register_function(ptycho)
print(func_uuid)

e68fff47-a540-4af0-aa61-fb7756a74c73


In [4]:
ifpath = '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/extracted_tekin/extracted_scan350.h5'
ofpath = '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/recons/funcx/01/'
script_path = '/home/bicer/projects/tike/scripts/mpi_tike-recon.py'
python_path = "/home/bicer/miniconda3/envs/ptycho/bin/python"

rec_alg = 'cgrad'
rec_nmodes = 8
rec_upd_pos = False
rec_niter = 30
rec_output_freq = 10
rec_recover_psi = True
rec_recover_probe= True
rec_recover_positions = False
rec_model = 'gaussian'
rec_ngpu = 8
rec_use_mpi = True

params = {'ifpath': ifpath,
          'ofpath': ofpath,
          'script_path': script_path,
          'python_path': python_path,
          'rec_alg': rec_alg,
          'rec_nmodes': rec_nmodes,
          'rec_upd_pos': rec_upd_pos,
         'rec_niter': rec_niter,
         'rec_output_freq': rec_output_freq,
         'rec_recover_psi': rec_recover_psi,
         'rec_recover_probe': rec_recover_probe,
         'rec_recover_positions': rec_recover_positions,
         'rec_model': rec_model,
         'rec_ngpu': rec_ngpu,
         'rec_use_mpi': rec_use_mpi}

theta_endpoint = 'f765db7a-038c-47ea-9176-d81de31c054f'
res = fxc.run(params, endpoint_id=theta_endpoint, function_id=func_uuid)
print(res)

f7f022d6-7c1b-49de-86fe-f7db9de4b7f9


In [5]:
while True:
    try:
        time.sleep(3) #3 secs
        mres = fxc.get_result(res)
        break
    except Exception as e:
        print(e)

waiting-for-ep
waiting-for-ep
waiting-for-ep
waiting-for-ep
waiting-for-ep
waiting-for-launch
waiting-for-launch
waiting-for-launch
waiting-for-launch
waiting-for-launch
waiting-for-launch
waiting-for-launch
waiting-for-launch
waiting-for-launch
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


In [6]:
print(mres)

b'scan range is (0, 648.7589111328125), (0, 1137.08203125).\nscan positions are (1, 1847, 2), float32\ndata is (1, 1847, 128, 128), float32\nprobe is (1, 1, 1, 7, 128, 128), complex64\n'
